In [2]:
import pandas as pd
import duckdb, pathlib

In [3]:
con = duckdb.connect("../data/processed/enron.duckdb")

In [7]:
# List all tables in the DuckDB database to find the correct table name
print(con.execute("SHOW TABLES").fetchdf())

     name
0  parsed


In [11]:
con.execute("""
COPY (SELECT * FROM parsed) TO '../data/processed/cleaned_emails.parquet' (FORMAT PARQUET);
""")


In [16]:
df = con.table("parsed").fetchdf()
print(df.head())

                      from                       to cc bcc    subject  \
0  phillip.allen@enron.com     tim.belden@enron.com                     
1  phillip.allen@enron.com  john.lavorato@enron.com               Re:   
2  phillip.allen@enron.com   leah.arsdall@enron.com          Re: test   
3  phillip.allen@enron.com    randall.gay@enron.com                     
4  phillip.allen@enron.com     greg.piper@enron.com         Re: Hello   

                       date                                         msg_id  \
0 2001-05-14 19:39:00-04:00  <18782981.1075855378110.JavaMail.evans@thyme>   
1 2001-05-04 16:51:00-04:00  <15464986.1075855378456.JavaMail.evans@thyme>   
2 2000-10-18 06:00:00-04:00  <24216240.1075855687451.JavaMail.evans@thyme>   
3 2000-10-23 09:13:00-04:00  <13505866.1075863688222.JavaMail.evans@thyme>   
4 2000-08-31 08:07:00-04:00  <30922949.1075863688243.JavaMail.evans@thyme>   

                                          clean_body  has_other_content  \
0                

In [15]:
# df = pd.read_parquet("../data/processed/cleaned_emails.parquet")
# print(df.head())

                      from                       to cc bcc    subject  \
0  phillip.allen@enron.com     tim.belden@enron.com                     
1  phillip.allen@enron.com  john.lavorato@enron.com               Re:   
2  phillip.allen@enron.com   leah.arsdall@enron.com          Re: test   
3  phillip.allen@enron.com    randall.gay@enron.com                     
4  phillip.allen@enron.com     greg.piper@enron.com         Re: Hello   

                       date                                         msg_id  \
0 2001-05-14 23:39:00+00:00  <18782981.1075855378110.JavaMail.evans@thyme>   
1 2001-05-04 20:51:00+00:00  <15464986.1075855378456.JavaMail.evans@thyme>   
2 2000-10-18 10:00:00+00:00  <24216240.1075855687451.JavaMail.evans@thyme>   
3 2000-10-23 13:13:00+00:00  <13505866.1075863688222.JavaMail.evans@thyme>   
4 2000-08-31 12:07:00+00:00  <30922949.1075863688243.JavaMail.evans@thyme>   

                                          clean_body  has_other_content  \
0                

In [18]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=None)
train.to_parquet("../data/processed/train_unlabeled.parquet")
test.to_parquet("../data/processed/test_unlabeled.parquet")


Now, we still need a small sample of, say, 800 emails that I will manually label and will train the baseline model in. Then will train the model in the remaining emails. I will manually label those emails that the model is the least confident in and i will add them to the training_labeled and remove them from the training_unlabeled, and repeat a couple times (perhaps 2-3).

In [35]:
train_unlabeled_0, train_to_label = train_test_split(train, test_size=0.002, random_state=42, stratify=None)
train_unlabeled_0.to_parquet("../data/processed/train_unlabeled_0.parquet") # has ~400k - 800 emails
train_to_label.to_parquet("../data/processed/train_to_label.parquet") # has 800 emails initially

Confirming the split correctly happened.

In [39]:
train_df = pd.read_parquet("../data/processed/train_unlabeled.parquet", engine='pyarrow')
train_un0_df = pd.read_parquet("../data/processed/train_unlabeled_0.parquet", engine='pyarrow')
train_tol_df = pd.read_parquet("../data/processed/train_to_label.parquet", engine='pyarrow')

In [40]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406342 entries, 421447 to 121958
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype                           
---  ------             --------------   -----                           
 0   from               406342 non-null  object                          
 1   to                 406342 non-null  object                          
 2   cc                 406342 non-null  object                          
 3   bcc                406342 non-null  object                          
 4   subject            406342 non-null  object                          
 5   date               406342 non-null  datetime64[us, America/New_York]
 6   msg_id             406342 non-null  object                          
 7   clean_body         406342 non-null  object                          
 8   has_other_content  406342 non-null  bool                            
 9   is_forwarded       406342 non-null  bool                            
d

In [41]:
train_un0_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405529 entries, 423984 to 243046
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype                           
---  ------             --------------   -----                           
 0   from               405529 non-null  object                          
 1   to                 405529 non-null  object                          
 2   cc                 405529 non-null  object                          
 3   bcc                405529 non-null  object                          
 4   subject            405529 non-null  object                          
 5   date               405529 non-null  datetime64[us, America/New_York]
 6   msg_id             405529 non-null  object                          
 7   clean_body         405529 non-null  object                          
 8   has_other_content  405529 non-null  bool                            
 9   is_forwarded       405529 non-null  bool                            
d

In [42]:
train_tol_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 813 entries, 448549 to 152527
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype                           
---  ------             --------------  -----                           
 0   from               813 non-null    object                          
 1   to                 813 non-null    object                          
 2   cc                 813 non-null    object                          
 3   bcc                813 non-null    object                          
 4   subject            813 non-null    object                          
 5   date               813 non-null    datetime64[us, America/New_York]
 6   msg_id             813 non-null    object                          
 7   clean_body         813 non-null    object                          
 8   has_other_content  813 non-null    bool                            
 9   is_forwarded       813 non-null    bool                            
dtypes: bool(2),

Now I will manually label the 800 emails.

In [67]:
pd.set_option('display.max_colwidth', None)
train_tol_df.head(2)

,from,to,cc,bcc,subject,date,msg_id,clean_body,has_other_content,is_forwarded,label
448549,kayne.coulter@enron.com,"jay.wills@enron.com, cyril.price@enron.com, john.kinser@enron.com, rudy.acevedo@enron.com, richard.hrabal@enron.com, juan.hernandez@enron.com, greg.trefz@enron.com, miguel.garcia@enron.com, russell.ballato@enron.com, joe.stepenovitch@enron.com, joe.errigo@enron.com, doug.miller@enron.com, larry.campbell@enron.com, keller.mayeaux@enron.com, chad.starnes@enron.com, dean.laurent@enron.com, don.baughman@enron.com, lawrence.clayton@enron.com, christopher.watts@enron.com, steve.olinde@enron.com, larry.jester@enron.com",,,Alternate Location for NYISO Schedules,2001-02-13 02:29:00-05:00,<19602093.1075851837161.JavaMail.evans@thyme>,,False,True,fwd_chain
306937,kay.mann@enron.com,christina.valdez@enron.com,,,Dick's aloha,2001-02-20 09:17:00-05:00,<9135302.1075845816031.JavaMail.evans@thyme>,"Hey there, I'm going to try very hard to make it to Dick's farewell party. Who did the invitations? It is so clever. Kay",False,False,


In [44]:
train_tol_df["label"] = ""

In [ ]:
train_tol_df.at[448549, "label"] = "logistics"
train_tol_df.at[306937, "label"] = "purely_personal"
train_tol_df.at[402914, "label"] = "purely_personal"
train_tol_df.at[469118, "label"] = "company_business"
train_tol_df.at[369161, "label"] = "company_business"
train_tol_df.at[32576, "label"] = "fwd_chain"


In [65]:
train_tol_df.loc[train_tol_df["is_forwarded"] == True, "label"] = "fwd_chain"

In [68]:
train_tol_df.to_parquet("../data/processed/train_to_label_fwd.parquet", engine='pyarrow')

Labeled all emails in the parquet file, which i could only download back a csv, so now we'll bring it back and rename it.

In [87]:
df = pd.read_csv("../data/processed/Labeled_Emails.csv")
df.to_parquet("../data/processed/train_labeled_813.parquet", engine='pyarrow')

In [88]:
train_labeled_0_df = pd.read_parquet("../data/processed/train_labeled_813.parquet", engine='pyarrow')
train_labeled_0_df_modified = train_labeled_0_df.drop(columns=["Unnamed: 0", "index", "text"], errors='ignore')
train_labeled_0_df_modified.set_index("__index_level_0__", inplace=True)
train_labeled_0_df_modified.index.name = None
train_labeled_0_df_modified.head()

,from,to,cc,bcc,subject,date,msg_id,clean_body,has_other_content,is_forwarded,label
448549,kayne.coulter@enron.com,"jay.wills@enron.com, cyril.price@enron.com, john.kinser@enron.com, rudy.acevedo@enron.com, richard.hrabal@enron.com, juan.hernandez@enron.com, greg.trefz@enron.com, miguel.garcia@enron.com, russell.ballato@enron.com, joe.stepenovitch@enron.com, joe.errigo@enron.com, doug.miller@enron.com, larry.campbell@enron.com, keller.mayeaux@enron.com, chad.starnes@enron.com, dean.laurent@enron.com, don.baughman@enron.com, lawrence.clayton@enron.com, christopher.watts@enron.com, steve.olinde@enron.com, larry.jester@enron.com",None,None,alternate location for nyiso schedules,2001-02-13T07:29:00-05:00,<19602093.1075851837161.JavaMail.evans@thyme>,None,False,True,fwd_chain
306937,kay.mann@enron.com,christina.valdez@enron.com,None,None,dick's aloha,2001-02-20T14:17:00-05:00,<9135302.1075845816031.JavaMail.evans@thyme>,"hey there, i'm going to try very hard to make it to dick's farewell party. who did the invitations? it is so clever. kay",False,False,purely_personal
402914,susan.scott@enron.com,monique.sanchez@enron.com,None,None,re: howdy,2000-10-27T13:37:00-04:00,<261394.1075846801224.JavaMail.evans@thyme>,"no need to apologize for keeping me out wednesday night, it was probably more a case of me keeping myself out (i have been known to do that on occasion). i'm sorry i didn't return your call yesterday, but i had a pretty crazy day here and that mixed with me only running on about two cylinders effectively cut me off from the outside world. as for being upset about anything, don't worry i'm not. you know me well enough to know that my actions are my own. i'm not sure if i'm going to stay in austin saturday night or not, but in case i do my cell # is 281-844-0866. susan i'll see you at purvi's happy hour if you end up going.",False,False,purely_personal
469118,dhudanish@brobeck.com,legal <.taylor@enron.com>,None,None,None,2001-10-26T00:12:37-04:00,<23252628.1075861969791.JavaMail.evans@thyme>,"mark, i am sure you are overwhelmed on other projects, but i wanted to make sure that you had received the attached. i have some comments / questions on the form of electronic trading agreement you sent me. no rush, just call me at your convenience. i hope you are well. nobel started this week -- a great excuse to go out for a nice lunch! regards, david original enron tou.doc ( microsoft word document ) revised enron terms of use.doc ( microsoft word document ) redline.doc ( word document ) < > < > < > ======================================================= this email message is for the sole use of the intended recipient(s) and may contain confidential and privileged information. any unauthorized review, use, disclosure or distribution is prohibited. if you are not the intended recipient, please contact the sender by reply email and destroy all copies of the original message. to reply to our email administrator directly, send an email to postmaster@brobeck.com brobeck phleger & harrison llp http://www.brobeck.com - original enron tou.doc - revised enron terms of use.doc - redline.doc",False,False,action_required
369161,benjamin.rogers@enron.com,dee.madole@enron.com,None,None,None,2000-03-29T20:30:00-05:00,<18633992.1075857161446.JavaMail.evans@thyme>,"here is a presentation that i need by 11:00am thursday. the presentation is almost complete, but if you can clean it up that would be great. i will need 8 coler copies and bound. please give me a call before you have someone run the copies, because we may have some early morning changes. thanks ben",False,False,action_required


In [89]:
train_labeled_0_df_modified.to_parquet("../data/processed/train_labeled_813.parquet", engine='pyarrow')

Now we delete the un-needed files and we train! Renamed this last modified file to "train_labeled_813" for its initial size.

In [ ]:
# Taking a look as a sanity check
df = pd.read_parquet("../data/processed/train_unlabeled_0.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405529 entries, 423984 to 243046
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype                           
---  ------             --------------   -----                           
 0   from               405529 non-null  object                          
 1   to                 405529 non-null  object                          
 2   cc                 405529 non-null  object                          
 3   bcc                405529 non-null  object                          
 4   subject            405529 non-null  object                          
 5   date               405529 non-null  datetime64[us, America/New_York]
 6   msg_id             405529 non-null  object                          
 7   clean_body         405529 non-null  object                          
 8   has_other_content  405529 non-null  bool                            
 9   is_forwarded       405529 non-null  bool                            
d

At this point I load the larger dataset (where I will be deleting the batched emails from) and labeled batched emails

In [182]:
full_df = pd.read_parquet("../data/processed/train_unlabeled_2.parquet", engine='pyarrow')
labeled_batch = pd.read_csv("../data/processed/Batch_Renamed_Emails_2.csv")

In [107]:
# full_df.head()

In [183]:
# labeled_batch.head()
print(labeled_batch.columns)
# labeled_batch = labeled_batch.iloc[0:]
labeled_batch.info()

Index(['orig_index', 'from', 'to', 'msg_id', 'cc', 'bcc', 'clean_body',
       'subject', 'date', 'has_other_content', 'is_forwarded', 'label'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   orig_index         200 non-null    int64 
 1   from               200 non-null    object
 2   to                 200 non-null    object
 3   msg_id             200 non-null    object
 4   cc                 6 non-null      object
 5   bcc                6 non-null      object
 6   clean_body         200 non-null    object
 7   subject            200 non-null    object
 8   date               200 non-null    object
 9   has_other_content  200 non-null    bool  
 10  is_forwarded       200 non-null    bool  
 11  label              200 non-null    object
dtypes: bool(2), int64(1), object(9)
memory usage: 16.1+ KB


Then I make the batch email dataset match the larger datasets I started with.

In [ ]:
labeled_batch.set_index("orig_index", inplace=True)
labeled_batch.index.name = None
labeled_batch.fillna("", inplace=True) 
labeled_batch.head()

from  \
18350   trimlife.weight.loss-return-11-7-10825134-x3pacl23neeotqbz-274@mta10.optamail.com   
503717                                                            bluechipbrands@m-ul.com   
257678                                                noreply@ccomad3.uu.commissioner.com   
97107                                                 noreply@ccomad3.uu.commissioner.com   
54101                                                 noreply@ccomad3.uu.commissioner.com   

                               to  \
18350      don.baughman@enron.com   
503717      jason.wolfe@enron.com   
257678  tori.kuykendall@enron.com   
97107       frank.ermis@enron.com   
54101     martin.cuilla@enron.com   

                                               msg_id cc bcc  \
18350   <20251621.1075840328743.JavaMail.evans@thyme>          
503717    <308892.1075861458224.JavaMail.evans@thyme>          
257678  <26584824.1075859025104.JavaMail.evans@thyme>          
97107   <28894847.1075861334622.JavaMail.evans@thyme>          
54101   <15901555.1075853086340.JavaMail.evans@thyme>          

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Then I create the next large dataset to work with in the next run. This dataset no longer contains the batch emails.

In [185]:
train_unlabeled_3 = full_df.drop(index=labeled_batch.index)

In [186]:
# print(train_unlabeled_1.loc[437173.0])
train_unlabeled_3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 404929 entries, 423984 to 243046
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype                           
---  ------             --------------   -----                           
 0   from               404929 non-null  object                          
 1   to                 404929 non-null  object                          
 2   cc                 404929 non-null  object                          
 3   bcc                404929 non-null  object                          
 4   subject            404929 non-null  object                          
 5   date               404929 non-null  datetime64[us, America/New_York]
 6   msg_id             404929 non-null  object                          
 7   clean_body         404929 non-null  object                          
 8   has_other_content  404929 non-null  bool                            
 9   is_forwarded       404929 non-null  bool                            
d

As a sanity check, I compare the counts of emails with the previous version of the dataset -- their difference should be the size of the batch emails!

In [187]:
# print(train_unlabeled_0.loc[437173.0])
train_unlabeled_2 = pd.read_parquet("../data/processed/train_unlabeled_2.parquet", engine='pyarrow')
train_unlabeled_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 405129 entries, 423984 to 243046
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype                           
---  ------             --------------   -----                           
 0   from               405129 non-null  object                          
 1   to                 405129 non-null  object                          
 2   cc                 405129 non-null  object                          
 3   bcc                405129 non-null  object                          
 4   subject            405129 non-null  object                          
 5   date               405129 non-null  datetime64[us, America/New_York]
 6   msg_id             405129 non-null  object                          
 7   clean_body         405129 non-null  object                          
 8   has_other_content  405129 non-null  bool                            
 9   is_forwarded       405129 non-null  bool                            
d

I save this new larger dataset with the name of the run. At this point I had gone through the learning loop 3 times.

In [188]:
train_unlabeled_3.to_parquet("../data/processed/train_unlabeled_3.parquet", engine='pyarrow')

Finally, I add the batch emails to the dataset I've been training the model with. Each dataset is labeled by including its row count.

In [189]:
df_1213 = pd.read_parquet("../data/processed/train_labeled_1213.parquet", engine='pyarrow')
df_1413 = pd.concat([df_1213, labeled_batch], ignore_index=True)
df_1413.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1414 entries, 0 to 1413
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   from               1413 non-null   object
 1   to                 1383 non-null   object
 2   cc                 802 non-null    object
 3   bcc                802 non-null    object
 4   subject            1381 non-null   object
 5   date               1413 non-null   object
 6   msg_id             1413 non-null   object
 7   clean_body         1327 non-null   object
 8   has_other_content  1413 non-null   object
 9   is_forwarded       1413 non-null   object
 10  label              1414 non-null   object
dtypes: object(11)
memory usage: 121.6+ KB


In [190]:
df_1413 = df_1413.drop_duplicates(subset=["msg_id"])
df_1413.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1414 entries, 0 to 1413
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   from               1413 non-null   object
 1   to                 1383 non-null   object
 2   cc                 802 non-null    object
 3   bcc                802 non-null    object
 4   subject            1381 non-null   object
 5   date               1413 non-null   object
 6   msg_id             1413 non-null   object
 7   clean_body         1327 non-null   object
 8   has_other_content  1413 non-null   object
 9   is_forwarded       1413 non-null   object
 10  label              1414 non-null   object
dtypes: object(11)
memory usage: 121.6+ KB


And done! Next I repeat all the above steps after giving the model this new larger training dataset.

In [191]:
df_1413.to_parquet("../data/processed/train_labeled_1413.parquet", engine='pyarrow')